# Natural Language Processing

Natural Language Processing (or NLP) is a discipline in computing that deals with the communication between natural (human) languages and computer languages. A common example of NLP is something like spellcheck or autocomplete. Essentially NLP is the field that focuses on how computers can understand and/or process natural/human languages.

### Recurrent Neural Networks

In this tutorial we will introduce a new kind of neural network that is much more capable of processing sequential data such as text or characters called a **recurrent neural network** (RNN).

We will learn how to use a RNN to do the following:
- Sentiment Analysis
- Character Generation

RNN's are complex and come in many different forms so in this tutorial we will focus on how they work and the kind of problems they are best suited for.

## Sequence Data

In the previous tutorials we focused on data that could represent as one static data point where the notion of time or step was irrelevant. Take for example our image data, it was simply a tensor of shape (width, height, channels). That data doesn't change or care about the notion of time.

In this tutorial we will look at sequences of text and learn how we can encode them in a meaningful way. Unlike images, sequence data such as long chains of text, weather patterns, videos and really anything where the notion of a step or time is relevant needs to be processed and handled in a special way.

But what do I mean by sequences and why is text data a sequence? Well that's a good question. Since textual data contains many words that follow in a very specific and meaningful order, we need to be able to keep track of each word and when it occurs in the data. Simply encoding say an entire paragraph of text into one data point wouldn't give us a very meaningful picture of the data and would be very difficult to do anything with. This is why we treat text as a sequence and process one word at a time. We will keep track of where each of these words appear and use that information to try to understand the meaning of pieces of text.

## 1. Encoding Text

As we know machine learning models and neural networks don't take raw text data as an input. This means we must somehow encode our textual data to umeric values that our models can understand. There are many different ways of doing this and we will look at a few examples below.

Before we get into the different encoding/preprocessing method, let's understand the information we can get from textual data by looking at the following two movide reviews.

`I though the movie was going to be bad, but is was actually amazing!`

`I though the movie was going to be amazing, but is was actually bad!`

Although these two sentences are very similar we know that they have very different meaning. This is because of the **ordering** of words, a very important property of textual data.

Now keep that in mind while we consider some different ways of encoding our textual data.

### 1.1 Bag of Words

The first and simplest way to encode our data is to use something called **bag of words.** This is a pretty easy technique where each word in a sentence is encoded with an integer and thrown into a collection that does not maintain the order of the word but does keep track of the frequency. Have a look at the python function below that encodes a string of text into bag of words.

In [6]:
vocab = {} # map word to integer representing it
word_encoding = 1
def bag_of_words(text):
  global word_encoding

  # create a list of all of the words in the text,
  # we'll assume there is no grammar in our text for this example
  words = text.lower().split(" ")

  # stores all of the encodings and their frequency
  bag = {}

  for word in words:
    if word in vocab:
      encoding = vocab[word] # get encoding from vocab
    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1

    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1

  return bag

text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


This isn't really the way we would do this in practice, but I hope it gives you an idea of how bag of words works. Notice that we've lost the order in which words appear. In fact, let's look at how this encoding works for the two entences we showed above.

In [7]:
positive_review = "I though the movie was going to be bad but is was actually amazing"
negative_review = "I though the movie was going to be amazing but is was actually bad"

pos_bag = bag_of_words(positive_review)
neg_bag = bag_of_words(negative_review)

print("Positive: ", pos_bag)
print("Negative: ", neg_bag)

Positive:  {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 17: 1, 18: 1, 2: 1, 19: 1, 20: 1}
Negative:  {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 20: 1, 18: 1, 2: 1, 19: 1, 17: 1}


We can see that even though these sentences have a very different meaning they are encoded exactly the same way. Obviously, this isn't going to fly. Let's look at some other methods.

### 1.2 Integer Encoding

The next technique we will look at is called **integer encoding.** This involves representing each word or character in a sentence as a unique integer and maintaining the order of these word. This should hopefully fix the problem we saw before where we lost the order of words.

In [11]:
vocab = {}
word_encoding = 1
def one_hot_encoding(text):
  global word_encoding

  words = text.lower().split(" ")
  encoding = []

  for word in words:
    if word in vocab:
      code = vocab[word]
      encoding.append(code)
    else:
      vocab[word] = word_encoding
      encoding.append(word_encoding)
      word_encoding += 1

  return encoding

text = "this is a test to see if this test will work is is test a a"
encoding = one_hot_encoding(text)
print(encoding)
print(vocab)

[1, 2, 3, 4, 5, 6, 7, 1, 4, 8, 9, 2, 2, 4, 3, 3]
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


And now let's have a look at one hot encoding on our movie reviews.

In [12]:
positive_review = "I though the movie was going to be bad but is was actually amazing"
negative_review = "I though the movie was going to be amazing but is was actually bad"

pos_encode = one_hot_encoding(positive_review)
neg_encode = one_hot_encoding(negative_review)

print("Positive: ", pos_encode)
print("Negative: ", neg_encode)

Positive:  [10, 11, 12, 13, 14, 15, 5, 16, 17, 18, 2, 14, 19, 20]
Negative:  [10, 11, 12, 13, 14, 15, 5, 16, 20, 18, 2, 14, 19, 17]


Much better, now we are keeping track of the order of the words and we can tell where each occurs. But this till has a few issues with it. Ideally when we encode words we would like similar words to have similar labels and different words to have very different labels. For example, the words happy and joyful should probably have a very similar labels so we can determine that they are similar. While words like horrible and amazing should be probably have very different labels. The method we looked at above won't be able to do something like this for use. This could mean that the model will have a very difficult time determining if two words are similar or not which could result in some pretty drastic performance impacts.

### 1.3 Word Embeddings

Luckily there is a third method that is far superior, **word embeddings.** This method keeps the order of words intact as well as encodes similar words with very similar labels. It attempts to not only encode the frequency and order of words but the meaning of those words in the sentence. It encodes each word as a dense vector that represents its context in the sentence.

Unlike the previous techniques word embeddings are learned by looking at many different training examples. You can add what's called an *embedding layer* to the beginning of your model and while your model trains your embedding layer will learn the correct embeddings for words. You can also use pretrained embedding layers.

This is the technique we will use for our examples and its implementation will be showed later on.

## 2. Recurrent Neural Network (RNN's)

Now we've learned a little bit about how we can encode text it's time to dive into recurrent neural networks. Up until this point we have been using something called **feed-forward** neural networks. This simply means that all our data is fed forwards (all  at once) from left to right through the network. This was fine for the problems we considered before but won't work very well for processing text. After all, even we (humans) don't process text all at once. We read word by word from left to right and keep track of the current meaning of the sentence so we can understand the meaning of the next word. Well this is exactly what a recurrent neural network is designed to do. When we say RNN all we really mean is network that contains a loop. A RNN will process one word at a time while maintaining an internal memory of what it's already seen. This will allow it to treat words differently based on their order in a sentence and to slowly build an understanding of the entire input, one word at a time.

This is why we are treating our text data as a sequence! So that we can pass one word at a time to the RNN.

Let's have a look at what a recurrent layer might look like.

![alt text](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)

Let's define what all these variables stand for before we get into the explination.

**h<sub>t</sub>** output at time t

**x<sub>t</sub>** input at time t

**A** Recurrent Layer (loop)

What this diagram is trying to illustrate is that a recurrent layer processes words or input one at a time in a combination with the output from the previous iteration. So, as we progress further in the input sequence, we build a more complex understanding of the text as a whole.

What we've just looked at is called a **simple RNN layer.** It can be effective at processing shorter sequences of text for simple problems but has many downfalls associated with it. One of them being the fact that as text sequences get longer it gets increasingly difficult for the network to understand the text properly.

## 3. LSTM

The layer we discussed in depth above was called a *simpleRNN.* However, there does exist some other recurrent layers (layers that contain a loop) that work much better than a simple RNN layer. The one we talk about here is called LSTM (Long Short-Term Memory). This layers works very similar to the simpleRNN layer but adds a way to access inputs from any timestamp in the past. Whereas in our simple RNN layer input from previous timestamps gradually disappeared as we got further through the input. With a LSTM we have a long-term memory data structure storing all the previously seen inputs as well as when we saw them. This allows for us to access any previous value we want at any point in time. This adds to the complexity of our network and allows it to discover more useful relationships between inputs and when they appear.

For the purpose of this course we will refrain
避免 from going any further into math or details behind how these layers work.

## 4. Sentiment情绪 Analysis

And now time to see a recurrent neural network in action. For example, we are going to do something called sentiment anlaysis.

The formal definition as follows:
*the process of computationally identifying and categorizing opinions expressed in a piece of text, especially in order to determine whether the writer's attitude towards a particular topic, product, etc. is positive, negative or neutral!*

The example we'll use here is classifying movie review as either positive, negative or neutral.

### 4.1 Movie Review Dataset

Well start by loading in the IMDB movie review dataset from keras. This dataset contains 25,000 reviews from IMDB where each one is already preprocessed and has a label as either positive or negative.. Each review is encoded by integers that represents how common a word is in the entire dataset. For example, a word encoded by the integer 3 means that it is the 3rd most common word in the dataset.

In [13]:
%tensorflow_version 2.x # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
17464789/17464789 [==============================] - 1s 0us/step


In [14]:
# let's look at one review
train_data[1]

[1,
 194,
 1153,
 194,
 8255,
 78,
 228,
 5,
 6,
 1463,
 4369,
 5012,
 134,
 26,
 4,
 715,
 8,
 118,
 1634,
 14,
 394,
 20,
 13,
 119,
 954,
 189,
 102,
 5,
 207,
 110,
 3103,
 21,
 14,
 69,
 188,
 8,
 30,
 23,
 7,
 4,
 249,
 126,
 93,
 4,
 114,
 9,
 2300,
 1523,
 5,
 647,
 4,
 116,
 9,
 35,
 8163,
 4,
 229,
 9,
 340,
 1322,
 4,
 118,
 9,
 4,
 130,
 4901,
 19,
 4,
 1002,
 5,
 89,
 29,
 952,
 46,
 37,
 4,
 455,
 9,
 45,
 43,
 38,
 1543,
 1905,
 398,
 4,
 1649,
 26,
 6853,
 5,
 163,
 11,
 3215,
 10156,
 4,
 1153,
 9,
 194,
 775,
 7,
 8255,
 11596,
 349,
 2637,
 148,
 605,
 15358,
 8003,
 15,
 123,
 125,
 68,
 23141,
 6853,
 15,
 349,
 165,
 4362,
 98,
 5,
 4,
 228,
 9,
 43,
 36893,
 1157,
 15,
 299,
 120,
 5,
 120,
 174,
 11,
 220,
 175,
 136,
 50,
 9,
 4373,
 228,
 8255,
 5,
 25249,
 656,
 245,
 2350,
 5,
 4,
 9837,
 131,
 152,
 491,
 18,
 46151,
 32,
 7464,
 1212,
 14,
 9,
 6,
 371,
 78,
 22,
 625,
 64,
 1382,
 9,
 8,
 168,
 145,
 23,
 4,
 1690,
 15,
 16,
 4,
 1355,
 5,
 28,
 6,
 52,
 

### 4.2 More Preprocessing

If we have a look at some of our loaded in reviews, we'll notice that they are different lengths. This is an issue. We cannot pass different length data into our neural network. Therefore,we must make each review the same length. To do this we will follow the procedure below:

- If the review is greater than 250 words then trim off the extra words
- If the review is less than 250 words ass the necessary amount of 0's to make it equal to 250.

Luckily for us keras has a function that can do this for us:

In [16]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

### 4.3 Creating the Model

Now it's time to create the model. We'll use a word embedding layer as the first layer in our model and add a LSTM layer afterwards that feeds into a dense node to get our predicted sentiment.

32 stands for the output dimension of the vectors generated by the embedding layer. We can change this value if we'd like!

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### 4.4 Training

Now it's time to compile and train the model

In [19]:
model.compile(loss="binary_crossentropy", optimizer='rmsprop', metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 76s 116ms/step - loss: 0.4673 - acc: 0.7704 - val_loss: 0.3604 - val_acc: 0.8478
Epoch 2/10
625/625 [==============================] - 70s 112ms/step - loss: 0.2704 - acc: 0.8942 - val_loss: 0.3510 - val_acc: 0.8630
Epoch 3/10
625/625 [==============================] - 70s 112ms/step - loss: 0.2095 - acc: 0.9237 - val_loss: 0.2841 - val_acc: 0.8874
Epoch 4/10
625/625 [==============================] - 68s 109ms/step - loss: 0.1654 - acc: 0.9416 - val_loss: 0.3181 - val_acc: 0.8868
Epoch 5/10
625/625 [==============================] - 75s 120ms/step - loss: 0.1354 - acc: 0.9538 - val_loss: 0.3096 - val_acc: 0.8814
Epoch 6/10
625/625 [==============================] - 68s 108ms/step - loss: 0.1132 - acc: 0.9622 - val_loss: 0.3526 - val_acc: 0.8844
Epoch 7/10
625/625 [==============================] - 74s 119ms/step - loss: 0.0954 - acc: 0.9689 - val_loss: 0.3484 - val_acc: 0.8666
Epoch 8/10
625/625 [==============================] - 7

And we'll evaluate the model on our training dataa to see how well it performs.

In [20]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 21s 27ms/step - loss: 0.4888 - acc: 0.8600
[0.4888279438018799, 0.8600000143051147]


So we're scoring somewhere inthe wid-high 80's. Not bad for a simple recurrent network.

### 4.5 Making Predictions

Now let's use our network to make predictions on our own reviews.

Since our reviews are encoded we'll need to convert any review that we write into that form so the network can understand it. To do that we'll load the encodings from the dataset and use them to encode our own data.

In [21]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

1641221/1641221 [==============================] - 1s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [22]:
# while were at it let's make a decode function

reversed_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
  PAD = 0
  text = ""
  for num in integers:
    if num != PAD:
      text += reversed_word_index[num] + " "

  return text[:-1]

print(decode_integers(encoded))

that movie was just amazing so amazing


In [30]:
# Now time to make a prediction
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print("Probability is positive: ", result[0])

positive_review = "That movie was great! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "That movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

1/1 [==============================] - 0s 42ms/step
Probability is positive:  [0.93666893]
1/1 [==============================] - 0s 51ms/step
Probability is positive:  [0.09865209]


---
## 5. RNN Play Generator

Now time for one of the coolest examples we've seen so far. We are going to use a RNN to generate a play. We will simply show the RNN and example of something we want it to recreate and it will learn how to write a version of it on its own. We'll do this using a character predictive model that will take as input a variable length sequence and predict the next character. We can use the model many times in a row with the output from the last prediction as the input for the next call to generate a sequence.

In [1]:
%tensorflow_version 2.x # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


### 5.1 Dataset

For this example, we only need one piece of training data. In fact, we can write our own poem or play and pass that to the network for training if we'd like. However, to make things easy we'll use an extract from a shakesphere play.

In [5]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

### 5.2 Loading Your Own Data

To load your own data, you'll need to upload a file from the dialog below. Then you'll need to follow the steps from above but load in this new file instead.

In [ ]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

### 5.3 Read Contents of File

Let's look at the contents of the file.

In [6]:
# Read, then decode for py2 compat
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [7]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



### 5.4 Encoding

Since this text isn't encoded yet we'll need to do that ourselves. We are going to encode each unique character as a different integer.

In [8]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [9]:
# let's look at how part of our text is encoded
print("Text: ", text[:13])
print("Encoded: ", text_to_int(text[:13]))

Text:  First Citizen
Encoded:  [18 47 56 57 58  1 15 47 58 47 64 43 52]


And here we will make a function that can convert our numeric values to text.

In [10]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


### 5.5 Creating Training Examples

Remember our task is to feed the model a sequence and have it return to use the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples.

The training example we will prepare will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to right. For example:

`input: Hell | output ello`

Our first step will be to create a stream of characters from our text data.

In [11]:
# length of sequence for a training example
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# create training examples / target
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

Next we can use the batch method to turn this stream of characters into batches of desired length.

In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

Now we need to use these sequences of length 101 and split them into input and output.

In [13]:
def split_input_target(chunk): # for example: hello
  input_text = chunk[:-1] # hell
  target_text = chunk[1:] # ello
  return input_text, target_text # hell, ello

# we use map to apply the above function to every entry
dataset = sequences.map(split_input_target)

In [14]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


Finally we need to make training batches.

In [15]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab) # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence memory. Instead,
# it maintains a buffer in which it shuffles elements)
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

### 5.6 Building the Model

Now it is time to build the model. We will use an embedding layer with LSTM and one dense layer that contains a node for each unique character in our training data. The dense layer will give us a probability distribution over all nodes.

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                batch_input_shape=[batch_size, None]),
      tf.keras.layers.LSTM(rnn_units,
                           return_sequences=True,
                           stateful=True,
                           recurrent_initializer='glorot_uniform'),
      tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)

### 5.7 Creating a Loss Function

Now we are going to create our own loss function for this problem. This is because our model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestamp for every sequence in the batch.

However, before we do that let's have a look at a sample input and the output from our untrained model. This is so we can understand what the model is giving us.

In [17]:
for input_example_batch, target_example_batch in data.take(1):
  # ask our model for a prediction on our first batch of training data (64 entries)
  example_batch_predictions = model(input_example_batch)
  # print out the output shape
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [18]:
# we can see that the prediction is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 3.3792574e-03  3.2805896e-04 -2.5893003e-04 ... -4.4132816e-04
   -4.4663884e-03 -1.5285050e-03]
  [ 2.8998875e-03 -6.7497324e-04  3.0139866e-03 ... -3.2255277e-03
   -6.5487223e-03 -3.0268591e-03]
  [ 2.4240511e-03 -2.1396074e-03  6.1180755e-03 ... -5.0955368e-03
   -8.3786473e-03 -4.0282854e-03]
  ...
  [ 2.3968085e-03  5.7009510e-03  5.5079930e-05 ...  2.5792769e-03
   -1.5958661e-03  2.7157767e-03]
  [ 4.9644806e-03  4.1875402e-03  4.8644003e-03 ...  3.8499525e-03
   -3.4208221e-03  1.9393467e-03]
  [ 8.2201064e-03 -7.9399208e-05  5.8706705e-03 ...  6.5194094e-05
   -1.0801763e-03  8.3084544e-03]]

 [[-2.2819163e-03  1.0318896e-02  3.0008554e-03 ... -1.5416979e-03
   -9.5761404e-04  2.1972810e-04]
  [-1.4730424e-03  5.8813617e-03  5.7368060e-03 ... -3.8139645e-03
   -2.6727044e-03 -2.1950058e-03]
  [-1.9655251e-03  2.0527847e-03  6.5448256e-03 ... -4.4233166e-04
    1.2373640e-03 -3.2310691e-03]
  ...
  [-6.6799857e-03 -5.2850256e-03  4.1440846e-03 ...  3.1125301e

In [19]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array
# is the prediction for the next character at each timestep

100
tf.Tensor(
[[ 3.3792574e-03  3.2805896e-04 -2.5893003e-04 ... -4.4132816e-04
  -4.4663884e-03 -1.5285050e-03]
 [ 2.8998875e-03 -6.7497324e-04  3.0139866e-03 ... -3.2255277e-03
  -6.5487223e-03 -3.0268591e-03]
 [ 2.4240511e-03 -2.1396074e-03  6.1180755e-03 ... -5.0955368e-03
  -8.3786473e-03 -4.0282854e-03]
 ...
 [ 2.3968085e-03  5.7009510e-03  5.5079930e-05 ...  2.5792769e-03
  -1.5958661e-03  2.7157767e-03]
 [ 4.9644806e-03  4.1875402e-03  4.8644003e-03 ...  3.8499525e-03
  -3.4208221e-03  1.9393467e-03]
 [ 8.2201064e-03 -7.9399208e-05  5.8706705e-03 ...  6.5194094e-05
  -1.0801763e-03  8.3084544e-03]], shape=(100, 65), dtype=float32)


In [20]:
# and finally we'll look at a prediction at the frst timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course it's 65 values representing the probability of each charater occuring next

65
tf.Tensor(
[ 3.3792574e-03  3.2805896e-04 -2.5893003e-04 -3.0964103e-03
 -1.0351235e-03  2.3863686e-03 -9.3210838e-04 -5.9830642e-04
  9.5150666e-04 -2.0009663e-03  4.9334746e-03  6.4832149e-03
 -2.0339254e-03 -5.1709386e-03 -1.1369063e-03 -5.0597903e-03
  5.0345771e-03  1.0114176e-02  4.8111053e-03  1.0312118e-03
  4.6595461e-03  1.9349949e-03 -2.3693307e-03 -1.7423134e-03
 -4.9118237e-03 -6.6490332e-04  1.0383390e-03  2.6730748e-03
 -2.2318354e-03 -7.9197586e-03 -5.9617567e-05 -4.9379207e-03
 -3.3375667e-03  1.3525317e-03 -3.4224866e-03  1.1605425e-03
 -3.7084986e-03  4.0121051e-03  7.5515057e-04 -2.5349895e-03
 -4.9129582e-04 -2.1795335e-04  1.0684133e-05  4.8142048e-03
 -2.2313218e-03 -3.0778227e-03  7.9146931e-03  1.1321430e-03
  1.6358790e-03  5.0885072e-03  3.3099554e-03  2.3764872e-03
 -1.5227303e-03  3.5902990e-03  6.9066654e-03 -1.4041360e-03
 -3.3462544e-03  1.7826961e-03  1.9458369e-03  4.1370434e-03
  2.5588016e-03  1.1718310e-03 -4.4132816e-04 -4.4663884e-03
 -1.528505

In [21]:
# If we want to determine the predicted character we need to sample
# the output distribution (pick a value based on probability)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to
# numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

# and this is what the model predicted for training sequence 1
predicted_chars

"3C!'D&c?JOXJmHy.I :qv;hmdkpxepNVtlb!wwTk3Ze-YIf.M;l'LF$jVKyYZvYJGI3FRNoUk''m:n$.mb&Ym-T3-a?HHY$HO,Yt"

So now we need to create a loss function that can compare that output to the expected output and give us some numeric value representing how close the two were.

In [22]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

### 5.8 Compiling the Model

At this point we can think of our problem as a classification problem where the model predicts the probability of each unique letter coming next.

In [23]:
model.compile(optimizer='adam', loss=loss)

### 5.9 Creating Checkpoints

Now we are going to setup and configure our model to save checkpoint as it trains. This will allow us to load our model from a checkpoint and continue training it.

In [24]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True
)

### 5.10 Training

Finally, we will start training the model.

**If this takeing a while go to Runtime > Change Runtime Type and choose "GPU" under hardware accelerator.**

In [26]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 17s 67ms/step - loss: 2.5950
Epoch 2/50
172/172 [==============================] - 12s 62ms/step - loss: 1.8892
Epoch 3/50
172/172 [==============================] - 13s 64ms/step - loss: 1.6372
Epoch 4/50
172/172 [==============================] - 14s 67ms/step - loss: 1.5019
Epoch 5/50
172/172 [==============================] - 14s 67ms/step - loss: 1.4207
Epoch 6/50
172/172 [==============================] - 14s 66ms/step - loss: 1.3641
Epoch 7/50
172/172 [==============================] - 14s 70ms/step - loss: 1.3182
Epoch 8/50
172/172 [==============================] - 14s 68ms/step - loss: 1.2782
Epoch 9/50
172/172 [==============================] - 14s 68ms/step - loss: 1.2429
Epoch 10/50
172/172 [==============================] - 13s 68ms/step - loss: 1.2056
Epoch 11/50
172/172 [==============================] - 13s 68ms/step - loss: 1.1698
Epoch 12/50
172/172 [==============================] - 14s 69ms/step - loss: 1.1328
E

### 5.11 Loading the Model

We'll rebuild the modell from a checkpoint using a batch_size of 1 so that we can feed one piece of text to the model and have it make a prediction.

In [27]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

Once the model is finishing training, we can find the **latest checkpoint** that stores the models weights using the following line.

In [29]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

We can load **any checkpoint** we want by specifying the exact file to load.

In [ ]:
checkpoint_num = 40
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

### 5.12 Genrating Text
Now we can use the lovely function provided by tensorflow to generate some text using any starting string we'd like.

In [32]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures reults is more predictable text
  # Higher temperatures results in more surprising text
  # Experiment to find the best setting
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)

    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # we pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [33]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: Hello
Hellow now.

QUEEN MARGARET:
Dive more, and fear me speak.

All Tonverse plumest
Retlessed upon the army weigh.

QUEEN MARGARET:
Why, how now, down! But what say you, Sir William Brandon,
For he be not a fevare--and they what you speak?
Why, ay: alas!
We would love her that he hath committed but Lucion
And here--but what I farewell.

GLOUCESTER:
But whenge there were tre enemies, besides that

GLOUCESTER:
I do suspect I have ever very well?
Who would be glad to see thee!

CORIOLANUS:
Scratches with braving art
Which you shall bear a poison,
Who thought a thousand thing is done;
Should show it me in socrary of a royal blessed guard,
To have a head, mother. I
have constant to wed: I say to thee, I beseech you
How but what I will charge thee, tell these headstrong well
The hearting of thy breath p


And that's pretty much it for this module! I highly reccomend messing with the model we just created and seeing what you can get it to do!